In [1]:
import numpy as np

In [2]:
def adj_d(a, b, adj): return adj[a, b]


def euc_2d(a, b): return np.linalg.norm(x=a-b, ord=2)


def att(a, b):
    d = euc_2d(a, b)
    d_ = float(int(d))

    if d_ < d:
        return d_ + 1.
    else:
        return d_

In [3]:
def make_genes(idxs, n):
    def make_gene(): return np.random.permutation(idxs)
    return [make_gene() for i in range(0, n)]


def ord_crossover(a, b, window=5, verbose=False):

    p1 = np.random.randint(0, len(a))

    p2 = (p1 + window) % len(a)

    l = len(a)

    if p2 < p1:
        p1, p2 = p2, p1

    o1 = a[p1:p2]
    o2 = b[p1:p2]

    if verbose:
        print(o1, o2)

    a = [e for e in a[p2:]+a[:p2] if e not in o2]
    b = [e for e in b[p2:]+b[:p2] if e not in o1]

    if verbose:
        print(b[l-p2:]+["o1"]+b[:l-p2], a[l-p2:]+["o2"]+a[:l-p2])

    o1 = b[l-p2:]+o1+b[:l-p2]
    o2 = a[l-p2:]+o2+a[:l-p2]

    return o1, o2


def disp_mutation(a, window=5, p=0.01, verbose=False):
    bit_mask = np.random.choice(a=[True, False], size=len(a), p=[p, 1-p])

    for i, mut in enumerate(bit_mask):
        if mut:
            p1 = i
            p2 = (p1 + window) % len(a)

            if p2 < p1:
                p1, p2 = p2, p1

            if verbose:
                print(a[p1: p2])

            a__ = [e for e in a if e not in a[p1: p2]]
            p3 = np.random.randint(0, len(a__))

            a = a__[:p3]+a[p1: p2]+a__[p3:]

    return a


def fitness(gene, adj, func):
    if func == adj_d:
        return sum([func(gene[i], gene[(i+1)%len(gene)], adj) for i in range(0, len(gene))])
    else:
        return sum([func(adj[gene[i]], adj[gene[(i+1)%len(gene)]]) for i in range(0, len(gene))])

In [6]:
class EvolutionTSP():
    def __init__(self, path, population_n, cross=ord_crossover, mutation=disp_mutation):
        self.cross = cross
        self.mutation = mutation
        self.adj = [10]*10
        self.fit_type = None

        self.population = make_genes(len(self.adj), population_n)
        pass

    def __call__(self):
        self.population = sorted(self.population, key=lambda x : fitness(x, self.adj, self.fit_type))
        l = len(self.population)
        l_4 = int(l/4)

        next_generation = []

        for i in range(0, l):
            g1 = np.random.choice(a=[0, 1, 2, 3], p=[0.5, 0.3, 0.15, 0.05])
            g2 = np.random.choice(a=[0, 1, 2, 3], p=[0.5, 0.3, 0.15, 0.05])
            p1 = g1 + np.random.randint(0, l_4)
            p2 = g2 + np.random.randint(0, l_4)
            o1, o2 = self.cross(self.population[p1], self.population[p2])
            next_generation.append(o1)
            next_generation.append(o2)
            
        self.population = map(self.muatation, next_generation)

In [7]:
tsp = EvolutionTSP(None, 10)

In [8]:
tsp.population

[array([6, 5, 3, 8, 4, 9, 2, 7, 1, 0]),
 array([4, 5, 3, 8, 2, 9, 1, 7, 6, 0]),
 array([7, 5, 0, 3, 2, 1, 8, 9, 4, 6]),
 array([0, 8, 9, 5, 7, 4, 1, 3, 2, 6]),
 array([1, 9, 7, 3, 0, 4, 5, 6, 8, 2]),
 array([1, 0, 7, 4, 8, 2, 9, 6, 5, 3]),
 array([6, 0, 5, 7, 1, 8, 3, 4, 2, 9]),
 array([1, 4, 5, 6, 8, 7, 0, 3, 9, 2]),
 array([0, 5, 3, 8, 2, 1, 6, 4, 9, 7]),
 array([0, 7, 4, 2, 3, 9, 1, 5, 8, 6])]

In [9]:
tsp()

TypeError: EvolutionTSP.__call__.<locals>.<lambda>() missing 1 required positional argument: 'y'